In [ ]:
#| default_exp passage

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#|hide
from nbdev import *
from fastcore.test import *
from fastcore.utils import *

# passage
> Reference API related to the passage ranking use case.

In [ ]:
#|export
import ir_datasets 
import json
import requests
import os 
import pandas as pd

from fastcore.utils import patch, patch_to
from typing import Dict, Tuple, Optional, List
import random
from vespa.package import (
    ApplicationPackage, 
    Field, 
    FieldSet, 
    RankProfile, 
    QueryField
)
from learntorank.query import QueryModel
from vespa.deployment import VespaDocker
from learntorank.evaluation import EvalMetric, evaluate
from learntorank.stats import compute_evaluation_estimates

## Data manipulation

Code related to the manipulation of passage ranking data.

In [ ]:
#| export
def sample_dict_items(
    d: Dict, # dict to be samples from.                     
    n: int   # Number of samples 
) -> Dict: # dict with sampled values
    "Sample items from a dict."
    n = min(len(d), n)
    return {
        k: d[k]
        for k in random.sample(
            k=n, population=sorted(d)
        )
    }

Usage:

In [ ]:
d = {"a": 1, "b":2, "c":3}

In [ ]:
sample_dict_items(d, 1)

{'a': 1}

In [ ]:
sample_dict_items(d, 2)

{'c': 3, 'b': 2}

In [ ]:
sample_dict_items(d, 3)

{'a': 1, 'c': 3, 'b': 2}

Return full dict in case number of samples is higher than length of the dict:

In [ ]:
sample_dict_items(d, 4)

{'a': 1, 'c': 3, 'b': 2}

In [ ]:
#|hide
test_eq(len(sample_dict_items(d, 4)), 3)

In [ ]:
#|export
def save_data(
    corpus: Dict,  # Document corpus, see usage example below. 
    train_qrels: Dict,  # Training relevance scores, see usage example below.  
    train_queries: Dict,  # Training queries, see usage example below.   
    dev_qrels: Dict,  #  Development relevance scores, see usage example below.  
    dev_queries: Dict,  # Development queries, see usage example below.   
    file_path: str = "passage_sample.json"  # Valid JSON file path.
) -> None:  # Side-effect: data is saved to `file_path`.
    """Save data to disk.
    
    The main goal is to save sample data to disk.
    """
    data = {
        "corpus": corpus, 
        "train_qrels": train_qrels, 
        "train_queries": train_queries, 
        "dev_qrels": dev_qrels, 
        "dev_queries": dev_queries
    }
    with open(file_path, "w") as f:
        json.dump(data, f)
    
    

Usage:

In [ ]:
corpus = {
    "0": "sentence 0", 
    "1": "sentence 1", 
    "2": "sentence 2", 
    "3": "sentence 3"
}
train_queries = {
    "10": "train query 10",
    "11": "train query 11"
}
train_qrels = {
    "10": {"0": 1},
    "11": {"2": 1}
}
dev_queries = {
    "20": "train query 20",
    "21": "train query 21"
}
dev_qrels = {
    "20": {"1": 1},
    "21": {"3": 1}
}

In [ ]:
save_data(
    corpus, 
    train_qrels, train_queries, 
    dev_qrels, dev_queries, 
    file_path="passage_sample.json"
)

In [ ]:
#|export
def load_data(
    file_path: Optional[str] = None  # valid JSON file path contain data saved by `save_data`. If `None`, a pre-generated sample will be downloaded.
) -> Dict:  # See usage example below for expected format.  
    """Load data.
    
    The main goal is to load sample data from disk.
    If a `file_path` is not provided, a pre-generated data sample 
    will be downloaded.
    """
    if not file_path:
        file_url = "https://data.vespa.oath.cloud/learntorank/passage/passage_sample.json"
        data_json = json.loads(
            requests.get(file_url).text
        )
    else:
        with open(file_path, "r") as f:
            data_json = json.load(f)
        
    return {
        "corpus": data_json["corpus"], 
        "train_qrels": data_json["train_qrels"], 
        "train_queries": data_json["train_queries"],
        "dev_qrels": data_json["dev_qrels"],
        "dev_queries": data_json["dev_queries"]
    }
    

Usage:

* With `file_path`:

In [ ]:
data = load_data("passage_sample.json")

In [ ]:
data

{'corpus': {'0': 'sentence 0',
  '1': 'sentence 1',
  '2': 'sentence 2',
  '3': 'sentence 3'},
 'train_qrels': {'10': {'0': 1}, '11': {'2': 1}},
 'train_queries': {'10': 'train query 10', '11': 'train query 11'},
 'dev_qrels': {'20': {'1': 1}, '21': {'3': 1}},
 'dev_queries': {'20': 'train query 20', '21': 'train query 21'}}

In [ ]:
#|hide
test_eq(
    data, 
    {
        "corpus": corpus, 
        "train_qrels": train_qrels, 
        "train_queries": train_queries, 
        "dev_qrels": dev_qrels, 
        "dev_queries": dev_queries
    }
)
os.remove("passage_sample.json")

* Without `file_path` specified, a pre-generated sample data will be downloaded:

In [ ]:
data = load_data()

In [ ]:
data.keys()

dict_keys(['corpus', 'train_qrels', 'train_queries', 'dev_qrels', 'dev_queries'])

In [ ]:
len(data["corpus"])

1000

In [ ]:
#|export 
class PassageData(object):
    def __init__(
        self, 
        corpus: Optional[Dict] = None,  # Document corpus, see usage example below.
        train_qrels: Optional[Dict] = None,  # Training relevance scores, see usage example below.  
        train_queries: Optional[Dict] = None,  # Training queries, see usage example below.   
        dev_qrels: Optional[Dict] = None,  #  Development relevance scores, see usage example below.  
        dev_queries: Optional[Dict] = None,  # Development queries, see usage example below.   
    ):
        "Container for passage data"
        self.corpus = corpus
        self.train_qrels = train_qrels
        self.train_queries = train_queries
        self.dev_qrels = dev_qrels
        self.dev_queries = dev_queries
    
    def __repr__(self):
        return "PassageData({}, {}, {}, {}, {})".format(
            "corpus" if self.corpus else None,
            "train_qrels" if self.train_qrels else None,
            "train_queries" if self.train_queries else None,
            "dev_qrels" if self.dev_qrels else None,
            "dev_queries" if self.dev_queries else None,            
        )
    
    def __eq__(self, obj: "PassageData"):
        return (self.corpus == obj.corpus and 
                self.train_qrels == obj.train_qrels and 
                self.train_queries == obj.train_queries and 
                self.dev_qrels == obj.dev_qrels and 
                self.dev_queries == obj.dev_queries)

Usage:

In [ ]:
corpus = {
    "0": "sentence 0", 
    "1": "sentence 1", 
    "2": "sentence 2", 
    "3": "sentence 3"
}
train_queries = {
    "10": "train query 10",
    "11": "train query 11"
}
train_qrels = {
    "10": {"0": 1},
    "11": {"2": 1}
}
dev_queries = {
    "20": "train query 20",
    "21": "train query 21"
}
dev_qrels = {
    "20": {"1": 1},
    "21": {"3": 1}
}

In [ ]:
passage_data = PassageData(
    corpus=corpus, 
    train_queries = train_queries, 
    train_qrels=train_qrels,
    dev_queries = dev_queries,
    dev_qrels = dev_qrels
)

In [ ]:
passage_data

PassageData(corpus, train_qrels, train_queries, dev_qrels, dev_queries)

In [ ]:
#|export
@patch
def save(self: PassageData, file_path: str = 'passage_sample.json'):
    save_data(
        corpus=self.corpus, 
        train_qrels=self.train_qrels, 
        train_queries=self.train_queries, 
        dev_qrels=self.dev_qrels, 
        dev_queries=self.dev_queries,
        file_path=file_path
    )

In [ ]:
#|hide
assert os.path.exists("passage_sample.json") == False, "File exists."

In [ ]:
passage_data.save()

In [ ]:
#|hide
assert os.path.exists("passage_sample.json") == True, "File does not exists."

In [ ]:
#|export
@patch_to(PassageData, cls_method=True)
def load(
    cls, file_path: Optional[str] = None  # valid JSON file path contain data saved by save_data. If None, a pre-generated sample will be downloaded.
) -> PassageData:
    "Load passage data from disk."
    data = load_data(file_path=file_path)
    return PassageData(
        corpus=data.get("corpus", None), 
        train_qrels=data.get("train_qrels", None), 
        train_queries=data.get("train_queries", None),
        dev_qrels=data.get("dev_qrels", None),
        dev_queries=data.get("dev_queries", None),        
    )
    

In [ ]:
data = PassageData.load(file_path="passage_sample.json")

In [ ]:
#|hide
test_eq(data, passage_data)
os.remove("passage_sample.json")

In [ ]:
data

PassageData(corpus, train_qrels, train_queries, dev_qrels, dev_queries)

In [ ]:
#|export
@patch(as_prop=True)
def summary(self: PassageData):
    "Summary of the size of the dataset components."
    print(
f"Number of documents: {len(self.corpus)}\n\
Number of train queries: {len(self.train_queries)}\n\
Number of train relevance judgments: {len(self.train_qrels)}\n\
Number of dev queries: {len(self.dev_queries)}\n\
Number of dev relevance judgments: {len(self.dev_qrels)}"
    )

In [ ]:
data.summary

Number of documents: 4
Number of train queries: 2
Number of train relevance judgments: 2
Number of dev queries: 2
Number of dev relevance judgments: 2


In [ ]:
#|hide 
test_stdout(lambda: data.summary, "Number of documents: 4\n\
Number of train queries: 2\n\
Number of train relevance judgments: 2\n\
Number of dev queries: 2\n\
Number of dev relevance judgments: 2")

In [ ]:
#|export 
@patch
def get_corpus(self: PassageData) -> pd.DataFrame:
    return pd.DataFrame([{"doc_id": k, "text": v} for k, v in self.corpus.items()])

In [ ]:
passage_data.get_corpus()

,doc_id,text
0,0,sentence 0
1,1,sentence 1
2,2,sentence 2
3,3,sentence 3


In [ ]:
#|hide
pd.testing.assert_frame_equal(
    passage_data.get_corpus(), 
    pd.DataFrame(data={"doc_id": [str(x) for x in range(4)], "text":[f"sentence {x}" for x in range(4)]})
)

In [ ]:
#|export 
@patch
def get_queries(
    self: PassageData, 
    type: str  # Either 'train' or 'dev'.
) -> pd.DataFrame:  # DataFrame conaining 'query_id' and 'query'.
    "Get query data."
    assert type in ['train', 'dev'], "'type' should be either 'train' or 'dev'."
    if type == 'train':
        data = self.train_queries
    elif type == 'dev':
        data = self.dev_queries
    return pd.DataFrame([{"query_id": k, "query": v} for k, v in data.items()])

In [ ]:
#|hide
test_fail(passage_data.get_queries, contains="get_queries() missing 1 required positional argument: 'type'")

In [ ]:
#|hide
test_fail(passage_data.get_queries, kwargs=dict(type="wrong_type"), contains="'type' should be either 'train' or 'dev'.")

In [ ]:
passage_data.get_queries(type="train")

,query_id,query
0,10,train query 10
1,11,train query 11


In [ ]:
#|hide
pd.testing.assert_frame_equal(
    passage_data.get_queries(type="train"), 
    pd.DataFrame(data={"query_id": [str(x + 10) for x in range(2)], "query":[f"train query {x + 10}" for x in range(2)]})
)

In [ ]:
passage_data.get_queries(type="dev")

,query_id,query
0,20,train query 20
1,21,train query 21


In [ ]:
#|hide
pd.testing.assert_frame_equal(
    passage_data.get_queries(type="dev"), 
    pd.DataFrame(data={"query_id": [str(x + 20) for x in range(2)], "query":[f"train query {x + 20}" for x in range(2)]})
)

In [ ]:
#|export 
@patch
def get_labels(
    self: PassageData, 
    type: str  # Either 'train' or 'dev'.
) -> Dict:  # pyvespa-formatted labeled data 
    "Get labeled data"
    assert type in ['train', 'dev'], "'type' should be either 'train' or 'dev'."
    if type == 'train':
        queries = self.train_queries
        qrels = self.train_qrels
    elif type == 'dev':
        queries = self.dev_queries
        qrels = self.dev_qrels        
    return [
        {
            "query_id": query_id, 
            "query": query, 
            "relevant_docs": [{"id": doc_id, "score": score} for doc_id, score in qrels[query_id].items()]
        } 
        for query_id, query in queries.items()
    ]

In [ ]:
#|hide
test_fail(passage_data.get_labels, contains="get_labels() missing 1 required positional argument: 'type'")

In [ ]:
#|hide
test_fail(passage_data.get_labels, contains="'type' should be either 'train' or 'dev'.", kwargs=dict(type="wrong_type"))

In [ ]:
passage_data.get_labels(type="train")

[{'query_id': '10',
  'query': 'train query 10',
  'relevant_docs': [{'id': '0', 'score': 1}]},
 {'query_id': '11',
  'query': 'train query 11',
  'relevant_docs': [{'id': '2', 'score': 1}]}]

In [ ]:
passage_data.get_labels(type="dev")

[{'query_id': '20',
  'query': 'train query 20',
  'relevant_docs': [{'id': '1', 'score': 1}]},
 {'query_id': '21',
  'query': 'train query 21',
  'relevant_docs': [{'id': '3', 'score': 1}]}]

In [ ]:
#|hide
test_eq(
    passage_data.get_labels(type="dev"), [
        {'query_id': '20',
         'query': 'train query 20',
         'relevant_docs': [{'id': '1', 'score': 1}]
        },
        {'query_id': '21',
         'query': 'train query 21',
         'relevant_docs': [{'id': '3', 'score': 1}]
        }
    ]
)

In [ ]:
#| export
def sample_data(
    n_relevant: int,  # The number of relevant documents to sample.
    n_irrelevant: int,  # The number of non-judged documents to sample. 
) -> PassageData:  

    """
    Sample data from the passage ranking dataset.
    
    The final sample contains `n_relevant` train relevant documents, 
    `n_relevant` dev relevant documents and `n_irrelevant` random documents
    sampled from the entire corpus. 
    
    All the relevant sampled documents, both from train and dev sets, 
    are guaranteed to be on the `corpus_sample`, which will contain 
    2 * `n_relevant` + `n_irrelevant` documents.
    """
    passage_corpus = ir_datasets.load("msmarco-passage")
    passage_train = ir_datasets.load("msmarco-passage/train/judged")    
    passage_dev = ir_datasets.load("msmarco-passage/dev/judged")
    
    train_qrels_dict = passage_train.qrels_dict()
    train_qrels_sample = sample_dict_items(d=train_qrels_dict, n=n_relevant)

    dev_qrels_dict = passage_dev.qrels_dict() 
    dev_qrels_sample = sample_dict_items(d=dev_qrels_dict, n=n_relevant)
    
    train_queries = {k:v for k,v in passage_train.queries_iter()}
    train_queries_sample = {k: train_queries[k] for k in train_qrels_sample.keys()}
    dev_queries = {k:v for k,v in passage_dev.queries_iter()}
    dev_queries_sample = {k: dev_queries[k] for k in dev_qrels_sample.keys()}

    train_positive_doc_id_samples = [
        doc_id[0]
        for doc_id in [list(docs.keys()) for docs in train_qrels_sample.values()]
    ]
    dev_positive_doc_id_samples = [
        doc_id[0]
        for doc_id in [list(docs.keys()) for docs in dev_qrels_sample.values()]
    ]

    docs_id = [x[0] for x in passage_corpus.docs_iter()]    
    docs_store = passage_corpus.docs_store()
    
    negative_doc_id_samples = random.sample(
        k=n_irrelevant, population=sorted(docs_id)
    )
    doc_id_samples = list(
        set(
            train_positive_doc_id_samples
            + dev_positive_doc_id_samples
            + negative_doc_id_samples
        )
    )
    corpus_sample = {k: docs_store.get(k)[1] for k in doc_id_samples}

    return PassageData(
        corpus=corpus_sample,
        train_qrels=train_qrels_sample,
        train_queries=train_queries_sample,
        dev_qrels=dev_qrels_sample,
        dev_queries=dev_queries_sample,
    )


Usage:

In [ ]:
#|eval:false
#|notest
sample = sample_data(n_relevant=1, n_irrelevant=3)

In [ ]:
#|hide
sample = PassageData.load(file_path="resources/passage/passage_sample.json")

The sampled corpus is a dict containing document id as key and the passage text as value.

In [ ]:
sample.corpus

{'890370': 'the map of europe gives you a clear view of the political boundaries that segregate the countries in the continent including germany uk france spain italy greece romania ukraine hungary austria sweden finland norway czech republic belgium luxembourg switzerland croatia and albaniahe map of europe gives you a clear view of the political boundaries that segregate the countries in the continent including germany uk france spain italy greece romania ukraine hungary austria sweden finland norway czech republic belgium luxembourg switzerland croatia and albania',
 '5060205': 'Setting custom HTTP headers with cURL can be done by using the CURLOPT_HTTPHEADER option, which can be set with the curl_setopt function. To add headers to your HTTP request you need to put them into a PHP Array, which you can then pass to the cul_setopt function, like demonstrated in the below example.',
 '6096573': "The sugar in RNA is ribose, whereas the sugar in DNA is deoxyribose. The only difference be

The size of the sampled document corpus is equal to 2 * `n_relevant` + `n_irrelevant`.

In [ ]:
len(sample.corpus)

5

In [ ]:
#|hide
test_eq(len(sample.corpus), 5)

Sampled queries are dict containing query id as key and query text as value.

In [ ]:
print(sample.train_queries)
print(sample.dev_queries)

{'899723': 'what sugar is found in rna'}
{'994205': 'which is the shortest stage in duration'}


Sampled qrels contains one relevant document for each query.

In [ ]:
print(sample.train_qrels)
print(sample.dev_qrels)

{'899723': {'6096573': 1}}
{'994205': {'7275560': 1}}


The following relevant documents are guaranteed to be included in the `corpus_sample`.

In [ ]:
#|echo:false
relevant_doc_ids= [
    x
    for query_id in sample.train_qrels 
    for x in sample.train_qrels[query_id]
] + [
    x 
    for query_id in sample.dev_qrels 
    for x in sample.dev_qrels[query_id]
]
relevant_doc_ids

['6096573', '7275560']

In [ ]:
#|hide
sample_corpus_ids = [x for x in sample.corpus]
assert all([x in sample_corpus_ids for x in relevant_doc_ids])

## Basic search

Code related to a basic search search engine for passage ranking.

In [ ]:
#|export
def create_basic_search_package(
    name: str="PassageRanking"  # Name of the application
) -> ApplicationPackage: # pyvespa [ApplicationPackage](https://pyvespa.readthedocs.io/en/latest/reference-api.html#applicationpackage) instance.
    """
    Create a basic Vespa application package for passage ranking.
    
    *Vespa fields*:
        
    The application contain two string fields: `doc_id` and `text`.
    
    *Vespa rank functions*:
    
    The application contain two rank profiles: 
    [bm25](https://docs.vespa.ai/en/reference/bm25.html) and
    [nativeRank](https://docs.vespa.ai/en/reference/nativerank.html).
    
    """
    app_package = ApplicationPackage(name=name)
    app_package.schema.add_fields(
        Field(name="doc_id", type="string", indexing=["attribute", "summary"]),
        Field(name="text", type="string", indexing=["index", "summary"], index="enable-bm25"),
    )
    app_package.schema.add_field_set(
        FieldSet(name="default", fields=["text"])
    )
    app_package.schema.add_rank_profile(
        RankProfile(
            name="bm25",
            first_phase="bm25(text)",
            summary_features=["bm25(text)"]
        )
    )
    app_package.schema.add_rank_profile(
        RankProfile(
            name="native_rank", 
            first_phase="nativeRank(text)"
        )
    )
    app_package.query_profile.add_fields(
        QueryField(name="maxHits", value=10000)
    )
    return app_package

Usage:

In [ ]:
app_package = create_basic_search_package(name="PassageModuleApp")

Check how the [Vespa schema definition](https://docs.vespa.ai/en/schemas.html) for this application looks like:

In [ ]:
print(app_package.schema.schema_to_text)

schema PassageModuleApp {
    document PassageModuleApp {
        field doc_id type string {
            indexing: attribute | summary
        }
        field text type string {
            indexing: index | summary
            index: enable-bm25
        }
    }
    fieldset default {
        fields: text
    }
    rank-profile bm25 {
        first-phase {
            expression: bm25(text)
        }
        summary-features {
            bm25(text)
        }
    }
    rank-profile native_rank {
        first-phase {
            expression: nativeRank(text)
        }
    }
}


## Evaluate query models

In [ ]:
#|export
def evaluate_query_models(
    app_package: ApplicationPackage, 
    query_models: List[QueryModel],
    metrics: List[EvalMetric],
    corpus_size: List[int],
    output_file_path: str,
    dev_query_percentage: float = 55578/8841823,
    verbose: bool = True,
    **kwargs
):

    print("*****")
    print("Deploy Vespa application:")
    print("*****")
    vespa_docker = VespaDocker(port=8183, cfgsrv_port=19173)
    app = vespa_docker.deploy(application_package=app_package)
    dfs = []
    for idx, n in enumerate(corpus_size):
        print("*****")
        print(f"Corpus size:{n}")
        print("*****")        
        if verbose:
            print("*****")
            print("Generate sample data:")
            print("*****")
        data = sample_data(
            n_relevant=int(n * dev_query_percentage), 
            n_irrelevant=int(n*(1-2*dev_query_percentage))
        )    
        if verbose:
            data.summary
            print("*****")
            print("Feed sample data to Vespa app:")
            print("*****")
        responses = app.feed_df(df=data.get_corpus(), include_id=True, id_field="doc_id", batch_size=10000)
        if verbose:
            print("*****")
            print("Evaluate query models")
            print("*****")
        labeled_data = data.get_labels(type="dev")
        assert len(labeled_data) > 0, "Need at least one query."
        evaluation_per_query = evaluate(
            app=app,
            labeled_data=labeled_data, 
            eval_metrics=metrics, 
            query_model=query_models, 
            id_field="doc_id",
            per_query=True,
            **kwargs
        )    
        estimates = compute_evaluation_estimates(
            df = evaluation_per_query
        )    
        estimates = estimates.assign(corpus_size=n, number_queries=len(labeled_data))
        if idx == 0:
            estimates.to_csv(output_file_path, index=False, mode="w")
        else:
            estimates.to_csv(output_file_path, index=False, mode="a", header=False)
        dfs.append(estimates)
        if verbose:
            print("*****")
            print("Delete all documents")
            print("*****")
        app.delete_all_docs(content_cluster_name="PassageRanking_content", schema="PassageRanking")
    if verbose:
        print("*****")
        print("Stop and remove Docker container")
        print("*****")
    vespa_docker.container.stop(timeout=600)
    vespa_docker.container.remove()
    estimates = pd.concat(dfs)
    return estimates

In [ ]:
from learntorank.evaluation import (
    MatchRatio,
    Recall, 
    ReciprocalRank, 
    NormalizedDiscountedCumulativeGain
)
from learntorank.query import QueryModel, OR, Ranking

corpus_size = [100, 200]
app_package = create_basic_search_package(name="PassageEvaluationApp")
query_models = [
    QueryModel(
        name="bm25", 
        match_phase=OR(), 
        ranking=Ranking(name="bm25")
    ),
    QueryModel(
        name="native_rank", 
        match_phase=OR(), 
        ranking=Ranking(name="native_rank")
    )
]
metrics = [
    MatchRatio(),
    Recall(at=100), 
    ReciprocalRank(at=10), 
    NormalizedDiscountedCumulativeGain(at=10)
]
output_file_path = "test.csv"

In [ ]:
estimates = evaluate_query_models(
    app_package=app_package,
    query_models=query_models,
    metrics=metrics,
    corpus_size=corpus_size,
    dev_query_percentage=0.5,
    output_file_path=output_file_path, 
    verbose=False
)

*****
Deploy Vespa application:
*****
Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for configuration server, 10/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Waiting for application status, 10/300 seconds...
Waiting for application status, 15/300 seconds...
Waiting for application status, 20/300 seconds...
Waiting for application status, 25/300 seconds...
Waiting for application status, 30/300 seconds...
Waiting for application status, 35/300 seconds...
Waiting for application status, 40/300 seconds...
Waiting for application status, 45/300 seconds...
Waiting for application status, 50/300 seconds...
Waiting for application status, 55/300 seconds...
Waiting for application status, 60/300 seconds...
Waiting for application status, 65/300 seconds...
Waiting for application status, 70/300 seconds...
Waiting for application status, 75/300 seconds...
Waiting fo

In [ ]:
#|hide
os.remove("test.csv")

In [ ]:
#|hide
nbdev_export()